In [8]:
from model_zoo import *
import pickle

In [3]:
data = pd.read_excel('./data/clean_df.xls')

In [4]:
X = data.copy().drop('label', axis = 1)
y = data.loc[:,'label']

In [5]:
## With a model from model_zoo, optimizes its hyperparamters with 10 fold CV and train the best estimator
## trained with the entire dataset. 

## Model is saved in $clf_name.sav pickle

In [7]:
model_dict = model_zoo[0]
param_grid = model_dict['param_grid']
model = model_dict['clf_fun']
model_name = model_dict['clf_name']

rkf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)

In [9]:
gridcv = RandomizedSearchCV(estimator = model, param_distributions= param_grid, scoring = 'roc_auc', cv = rkf,
                            refit = True, return_train_score = True, n_jobs = 4, random_state = 0, n_iter = 20, verbose = 1)

In [10]:
gridcv.fit(X,y)

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 out of  40 | elapsed:    3.2s remaining:    0.6s
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:    3.3s finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=0, shuffle=True),
          error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('mv', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('fs', SelectKBest(k=10, score_func=<function f_classif at 0x00000206D7136950>)), ('clf', LogisticRegression(C=1.0, class_weight=None, ...   penalty='l2', random_state=0, solver='warn', tol=0.0001,
          verbose=0, warm_start=False))]),
          fit_params=None, iid='warn', n_iter=20, n_jobs=4,
          param_distributions={'fs__k': [2, 4], 'clf__C': [0.1, 1]},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score=True, scoring='roc_auc', verbose=1)

In [18]:
best_model = gridcv.best_estimator_
filename = './models/' + model_name + '_model.sav'

In [19]:
pickle.dump(best_model, open(filename, 'wb'))

In [25]:
np.array(list(X))[best_model.named_steps['fs'].get_support(indices = True)]

array(['diabetes', 'angio.illvess2', 'timi_post', 'lab.cho.a'],
      dtype='<U19')